# 6) SageMaker pipeline
In this notebook, we will create a sagemaker pipeline to automate the process of training a model, and updating the deployed endpoint if it perform better on a given test set.  
Also slighty depricated, this documentation helped a lot:
https://docs.aws.amazon.com/sagemaker/latest/dg/define-pipeline.html

In [20]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.pipeline import Pipeline
from PIL import Image
import io
import base64
import json
import pprint
import time


session = sagemaker.Session()

bucket = session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

prefix = "capstone-inventory-project"
model_package_group_name = "Capstone_pipeline"

Default Bucket: sagemaker-us-east-1-646714458109
AWS Region: us-east-1
RoleArn: arn:aws:iam::646714458109:role/service-role/AmazonSageMaker-ExecutionRole-20211122T183493


In [21]:
import os
os.environ

environ{'REGION_NAME': 'us-east-1',
        'HOSTNAME': 'datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395',
        'CONDA_MD5': 'd63adf39f2c220950a063e0529d4ff74',
        'HOME': '/root',
        'CONDA_VERSION': 'py38_4.8.3',
        'AWS_CONTAINER_CREDENTIALS_RELATIVE_URI': '/_sagemaker-instance-credentials/572670c2f897621dcafe3612dc48a280feb07488198d3290b2e2ff49c1d0bada',
        'PYTHONNOUSERSITE': '0',
        'AWS_DEFAULT_REGION': 'us-east-1',
        'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/amazon/bin:/tmp/miniconda3/condabin:/tmp/anaconda3/condabin:/tmp/miniconda2/condabin:/tmp/anaconda2/condabin',
        'LANG': 'C.UTF-8',
        'AWS_ACCOUNT_ID': '646714458109',
        'DEBIAN_FRONTEND': 'noninteractive',
        'SHELL': '/bin/bash',
        'AWS_REGION': 'us-east-1',
        'AWS_INTERNAL_IMAGE_OWNER': 'Studio',
        'CONDA_DIR': '/opt/.sagemakerinternal/conda',
        'LC_ALL': 'C.UTF-8',
        'PWD': '/r

### 1) Define Pipeline Parameters

training_instance_type – The ml.* instance type of the training jobs.

input_data – The Amazon S3 location of the input data.

In [22]:
training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

input_data = ParameterString(
    name="InputData",
    default_value="s3://{}/{}/data".format(bucket, prefix)
)

# We will also define the were our model artefacts and evaluation results will be stored.
model_path = "s3://{}/{}/pipeline_model".format(bucket, prefix)
evaluation_file_path = "s3://{}/{}/current_accuracy.json".format(bucket, prefix)

### 1) Training step

In [23]:
estimator = PyTorch(
    entry_point="pipeline/1_train_pipeline.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.xlarge",        
    output_path = model_path,  # The training jobs output (mainly model artefacts) will go there.
    hyperparameters={                                        
        "batch-size": 64,
        "lr": 0.01}
)

In [24]:
step_train = TrainingStep(
    name="Training",
    estimator=estimator,
    inputs=input_data
)

### 2) Evaluation step

In [25]:
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.8",
    py_version="py36",
    instance_type=training_instance_type,
    image_scope="training"
)

In [26]:
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name="pipeline-eval",
    role=role
    #env= {"MODEL_DATA": step_train.properties.ModelArtifacts.S3ModelArtifacts}
)

In [27]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
last_report = PropertyFile(
    name="LastReport",
    output_name="evaluation",
    path="current_accuracy.json"
)

step_eval = ProcessingStep(
    name="Evaluation",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source="s3://sagemaker-us-east-1-646714458109/capstone-inventory-project/data/test/",
            destination="/opt/ml/processing/test"
        ),
        ProcessingInput(
            source=evaluation_file_path,
            destination="/opt/ml/processing/accuracy"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="pipeline/2_evaluation.py",
    property_files=[evaluation_report, last_report]
)

### 3) Update endpoint

In [28]:
script_update = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name="update-endpoint",
    role=role,
    env= {"MODEL_DATA": step_train.properties.ModelArtifacts.S3ModelArtifacts}
)

In [29]:
step_update = ProcessingStep(
    name="Update-endpoint",
    processor=script_update,
    inputs=[
        ProcessingInput(
            source="{}/evaluation.json".format(step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]),
            destination="/opt/ml/processing/accuracy"
        )
    ],
    code="pipeline/3_update_endpoint.py"
)

### 4) Condition step

In [30]:
# False step
script_else = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="else",
    role=role,
)

step_fail = ProcessingStep(
    name="Too-bad",
    processor=script_else,
    code="pipeline/3_else.py"
)

In [31]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name="Evaluation",
        property_file=evaluation_report,
        json_path="test_accuracy"
    ),
    right=JsonGet(
        step_name="Evaluation",
        property_file=last_report,
        json_path="Accuracy_of_deployed_model"
    )
)

In [32]:
step_cond = ConditionStep(
    name="Is-the-new_model-better-than-the-one-deployed",
    conditions=[cond_gte],
    if_steps=[step_update],  # step_register
    else_steps=[step_fail]
)

### 5) Pipeline creation

In [33]:
pipeline_name = "CapstonePipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,
        input_data
    ],
    steps=[step_train, step_eval, step_cond]
)

In [34]:
json.loads(pipeline.definition())

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-646714458109/capstone-inventory-project/data'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Training',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecification': {'TrainingInputMode': 'File',
     'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8-cpu-py36',
     'EnableSageMakerMetricsTimeSeries': True},
    'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-646714458109/capstone-inventory-project/pipeline_model'},
    'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
    'ResourceConfig': {'InstanceCount': 1,
     'InstanceType': 'ml.m5.xlarge',
     'VolumeSizeInGB': 30},
    '

In [35]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:646714458109:pipeline/capstonepipeline',
 'ResponseMetadata': {'RequestId': 'b5058e6c-06f0-40ba-b32b-f515bd077295',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b5058e6c-06f0-40ba-b32b-f515bd077295',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Fri, 21 Jan 2022 17:34:09 GMT'},
  'RetryAttempts': 0}}

In [70]:
#pipeline.delete()

In [35]:
#execution = pipeline.start()

In [45]:
execution.list_steps()

[{'StepName': 'Evaluation',
  'StartTime': datetime.datetime(2022, 1, 20, 14, 7, 16, 413000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 1, 20, 14, 13, 23, 770000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'AttemptCount': 0,
  'FailureReason': 'ClientError: Cannot access S3 key.',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:processing-job/pipelines-1kfl6nqdpjnj-evaluation-2dvfanug7b'}}},
 {'StepName': 'Training',
  'StartTime': datetime.datetime(2022, 1, 20, 13, 58, 57, 172000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 1, 20, 14, 7, 15, 617000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:training-job/pipelines-1kfl6nqdpjnj-training-4sxzkheorc'}}}]

In [42]:
execution.list_steps()[0]["Metadata"]

{'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:processing-job/pipelines-1kfl6nqdpjnj-evaluation-2dvfanug7b'}}

In [42]:
pytorch_container = sagemaker.image_uris.retrieve(framework="pytorch",
                                                  region=session.boto_region_name,
                                                  version="1.5",
                                                  instance_type="ml.m5.xlarge",
                                                  accelerator_type='ml.eia2.medium')    

Elastic inference is for inference only. Ignoring image scope: None.


In [51]:
# Remember that a model needs to have a unique name
model_name = "capstone-inventory-monitoring-model-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# We also need to tell SageMaker which container should be used for inference and where it should
# retrieve the model artifacts from. In our case, the xgboost container that we used for training
# can also be used for inference and the model artifacts come from the previous call to fit.
primary_container = {
    "Image": pytorch_container,
    "ModelDataUrl": "s3://sagemaker-us-east-1-646714458109/capstone-inventory-project/pipeline_model/pipelines-dfgli4j5dpf7-Training-Bm3kuLKLcG/output/model.tar.gz"
}

# And lastly we construct the SageMaker model
model_info = session.sagemaker_client.create_model(
                                ModelName = model_name,
                                ExecutionRoleArn = role,
                                #PrimaryContainer = primary_container
                                PrimaryContainer = pytorch_model)

ParamValidationError: Parameter validation failed:
Invalid type for parameter PrimaryContainer, value: <sagemaker.pytorch.model.PyTorchModel object at 0x7fb23c778410>, type: <class 'sagemaker.pytorch.model.PyTorchModel'>, valid types: <class 'dict'>

In [47]:
# As before, we need to give our endpoint configuration a name which should be unique
endpoint_config_name = "capstone-endpoint-config-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#Let's also create a data capture 
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/{prefix}/data_capture"
)


# And then we ask SageMaker to construct the endpoint configuration
endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = endpoint_config_name,
                            ProductionVariants = [{
                                "InstanceType": "ml.m5.xlarge",
                                "AcceleratorType": "ml.eia2.medium",
                                "InitialInstanceCount": 1,
                                "ModelName": model_name,
                                "VariantName": "Pytorch-model"
                            }],
                            DataCaptureConfig = { 
                                "CaptureOptions": [
                                    {
                                        "CaptureMode": "Input"
                                    },
                                    {
                                        "CaptureMode": "Output"
                                    }],
                            "DestinationS3Uri": "s3://sagemaker-us-east-1-646714458109/capstone-inventory-project/data_capture",
                            "EnableCapture": True,
                            "InitialSamplingPercentage": 100}
)




In [53]:
# Create and upload model
pytorch_model = PyTorchModel(model_data="s3://sagemaker-us-east-1-646714458109/capstone-inventory-project/main_training/pytorch-training-2022-01-17-10-08-10-837/output/model.tar.gz", 
                             role=role, 
                             py_version='py3',
                             framework_version='1.5')

#saved_model = pytorch_model.register()

In [54]:
predictor = pytorch_model.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.large'
                                )  

------!

In [ ]:
client = boto3.client('lambda')
response = client.invoke(
    FunctionName='inference_capstone',
    Payload='{"url": "https://sagemaker-us-east-1-646714458109.s3.amazonaws.com/capstone-inventory-project/data/train/1/00014.jpg"}',
)

In [ ]:
json.loads(response['Payload'].read().decode())["body"]

In [ ]:
pytorch-inference-2022-01-21-10-56-08-969

In [50]:
session.sagemaker_client.update_endpoint(EndpointName="pytorch-inference-2022-01-21-10-56-08-969", EndpointConfigName=endpoint_config_name)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:646714458109:endpoint/pytorch-inference-2022-01-21-10-56-08-969',
 'ResponseMetadata': {'RequestId': '6f6682a0-7ca9-4945-889a-de0db043a5ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f6682a0-7ca9-4945-889a-de0db043a5ad',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Fri, 21 Jan 2022 11:02:10 GMT'},
  'RetryAttempts': 0}}

In [5]:
print("Hello")
!cp file_list.json scripts/file_list.json

Hello


In [6]:
import os

os.mkdir("scripts/file")